In [1]:
# Import Dependencies
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
import pandas as pd

In [2]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [3]:
from sqlalchemy import Column, Integer, String, Float

In [5]:
# Create Database Connection
database_path = "project2.sqlite"
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [6]:
# Solongo's data table 
class grad_capital_income(Base):
    __tablename__ = "state_table"
    state_code = Column(String, primary_key=True)
    percent_hs_grad = Column(Float)
    per_capita_income = Column(Integer)

In [7]:
# Liz's data table 
class crime(Base):
    __tablename__ = "crime_table"
    state_code = Column(String, primary_key=True)
    homicide_rate = Column(Float)
    firearms_death_rate = Column(Float)
    total_firearm_deaths = Column(Integer)   

In [8]:
# Liz's poverty data table
class poverty(Base):
    __tablename__ = "poverty_table"
    state_code = Column(String, primary_key=True)
    sixteen_average = Column(Float)
    sixteen_error  = Column(Float)
    fifteen_average  = Column(Float)
    fifteen_error  = Column(Float)
    seventeen_average  = Column(Float)
    seventeen_error  = Column(Float)
    three_year_error  = Column(Float)

In [9]:
# Create the table Jule's data table 
class household(Base):
    __tablename__ = "household_table"
    state_code = Column(String, primary_key=True)
    household_size = Column(Float)
    median_income = Column(Integer)

In [10]:
# Create the table Whitney's data table 
class nonemp(Base):
    __tablename__ = "nonemp"
    state_code = Column(String, primary_key=True)
    county = Column(Integer)
    NRCPTOT = Column(Integer)

In [12]:
# Create a session
session = Session(bind=engine)

In [13]:
# Create a "Metadata" Layer That Abstracts our SQL Database
Base.metadata.create_all(engine)

In [14]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [15]:
# view all the classes that automap found
Base.classes.keys()

['crime_table', 'household_table', 'nonemp', 'poverty_table', 'state_table']

In [20]:
crime_table = Base.classes.crime_table
household_table = Base.classes.household_table
nonemp = Base.classes.nonemp
poverty_table = Base.classes.poverty_table
state_table = Base.classes.state_table

In [21]:
# create our session (link) from Python to the DB
session = Session(engine)

In [22]:
# A Inspector object 
inspector = inspect(engine)
# -----------------------------------------------------------

# Define fonts 
class color:
    BOLD = '\033[1m'
    END = '\033[0m'
# -----------------------------------------------------------    

# check how tables structured and described state table
state = inspector.get_columns('state_table')
print(color.BOLD + '\nstate_table' + color.END)

for c in state:
    print(c['name'], c['type'])

# -----------------------------------------------------------    
# check how tables structured and described state crime
crime = inspector.get_columns('crime_table')
print(color.BOLD + 'crime_table' + color.END)
for c in crime:
    print(c['name'], c['type'])  
    
# -----------------------------------------------------------  
# check how tables structured and described state poverty 
poverty  = inspector.get_columns('poverty_table')
print(color.BOLD + 'poverty_table1' + color.END)
for c in poverty:
    print(c['name'], c['type'])

# -----------------------------------------------------------  
# check how tables structured and described state household 

household  = inspector.get_columns('household_table')
print(color.BOLD + 'household_table' + color.END)
for c in household:
    print(c['name'], c['type'])

# -----------------------------------------------------------  
# check how tables structured and described state household 
nonemp1 = inspector.get_columns('nonemp')
print(color.BOLD + 'nonemp' + color.END)
for c in nonemp1:
    print(c['name'], c['type'])


state_table
state_code VARCHAR
percent_hs_grad FLOAT
per_capita_income INTEGER
crime_table
state_code VARCHAR
homicide_rate FLOAT
firearms_death_rate FLOAT
total_firearm_deaths INTEGER
poverty_table1
state_code VARCHAR
sixteen_average FLOAT
sixteen_error FLOAT
fifteen_average FLOAT
fifteen_error FLOAT
seventeen_average FLOAT
seventeen_error FLOAT
three_year_error FLOAT
household_table
state_code VARCHAR
household_size FLOAT
median_income INTEGER
nonemp
state_code VARCHAR
county INTEGER
NRCPTOT INTEGER


## Insert data


In [25]:
# Load state data from csv
state_data = pd.read_csv("data/state_table.csv")
state_data=state_data.values.tolist()

# delete existing data so we can re-populate it
conn.execute("delete from state_table")

# insert rows
for row in state_data:
    conn.execute(f"""
    insert into state_table (state_code, percent_hs_grad, per_capita_income) 
    values ('{row[0]}', {row[1]}, {row[2]})
""")
    
# validate count
state_table_data = session.query(state_table).all()

print(f"Source CSV: {len(state_data)} records")
print(f"Table: {len(state_table_data)} records")



Source CSV: 51 records
Table: 51 records


In [26]:
# Load crime data
! head data/crime_table.csv


# Load crime data from csv
crime_data = pd.read_csv("data/crime_table.csv")
crime_data = crime_data.values.tolist()

# delete existing data so we can re-populate it
conn.execute("delete from crime_table")

# insert rows
for row in crime_data:    
    conn.execute(f"""
    insert into crime_table (state_code, homicide_rate, firearms_death_rate, total_firearm_deaths) 
    values ('{row[0]}', {row[1]}, {row[2]}, {row[3]})
""")
    
# validate count
crime_table_data = session.query(crime_table).all()

print(f"Source CSV: {len(crime_data)} records")
print(f"Table: {len(crime_table_data)} records")

State,Homicide Rate,Firearms Death Rate,Total Firearm Deaths
Louisiana,12.4,21.3,987
Missouri,9.8,19.0,1144
Nevada,9.1,16.8,498
Maryland,9.0,11.9,707
Arkansas,8.6,17.8,541
Alaska,8.4,23.3,177
Alabama,8.3,21.5,1046
Mississippi,8.2,19.9,587
Tennessee,7.8,17.1,1148
Source CSV: 50 records
Table: 50 records


In [27]:
# Load household data
! head data/household_table.csv


# Load household data from csv
household_data = pd.read_csv("data/household_table.csv")
household_data = household_data.values.tolist()

# delete existing data so we can re-populate it
conn.execute("delete from household_table")

# insert rows
for row in household_data:    
    conn.execute(f"""
    insert into household_table (state_code, household_size, median_income) 
    values ('{row[0]}', {row[1]}, {row[2]})
""")
    
#validate count
# household_table_data = session.query(household_table).all()

# print(f"Source CSV: {len(household_data)} records")
# print(f"Table: {len(household_table_data)} records")

In [38]:
# Load poverty data
! head data/poverty_table.csv


# Load poverty data from csv
poverty_data = pd.read_csv("data/poverty_table.csv")
poverty_data = poverty_data.values.tolist()
#for item in poverty_data:
#    print(poverty_data)

#  delete existing data so we can re-populate it
conn.execute("delete from poverty_table")

# insert rows
for row in poverty_data:
    sql = f"""
    insert into poverty_table (state_code, sixteen_average, sixteen_error, fifteen_average, 
                                fifteen_error, seventeen_average, seventeen_error, three_year_error) 
    values ('{row[0]}', {row[1]}, {row[2]}, {row[3]}, {row[4]}, {row[5]}, {row[6]}, {row[7]})
"""
    # print(sql)
    conn.execute(sql)
    
#validate count
poverty_table_data = session.query(poverty_table).all()
print(f"Source CSV: {len(poverty_data)} records")
print(f"Table: {len(poverty_table_data)} records")

state_code,sixteen_average,sixteen_error,fifteen_average,fifteen_error,seventeen_average,seventeen_error,three_year_error
Alabama,16,1.5,16.3,1.9,15.6,1.6,1.9
Alaska,12.2,1.8,10.9,1.6,12.6,1.6,2
Arizona,14.4,1.6,16.6,1.5,13.2,1.8,2
Arkansas  ,15.5,1.1,16.1,1.1,15.4,1.4,1.8
California,12.5,0.5,13.9,0.6,12,0.6,0.8
Colorado,8.9,0.9,9.2,1.4,9,1.1,1.7
Connecticut,9.9,1.3,9.5,1.4,10.4,1.4,1.8
Delaware,9.6,1.1,11.3,1.4,8.1,1.2,1.8
District of Columbia,14.9,1.1,16.5,1.3,14.3,1.2,1.7
Source CSV: 51 records
Table: 50 records


In [42]:
# Load household data
#! head data/nonemp.csv


# Load household data from csv
nonemp_data = pd.read_csv("data/nonemp.csv")
nonemp_data = nonemp_data.values.tolist()

print(nonemp_data)

# # delete existing data so we can re-populate it
conn.execute("delete from nonemp")

# nonemp1
# state_code VARCHAR
# county INTEGER
# NRCPTOT INTEGER


# insert rows
for row in nonemp_data:    
    conn.execute(f"""
    insert into nonemp (state_code, county, NRCPTOT) 
    values ('{row[0]}', {row[1]}, {row[2]})
""")
    
#validate count
nonemp_table_data = session.query(nonemp).all()

print(f"Source CSV: {len(nonemp_data)} records")
print(f"Table: {len(nonemp_table_data)} records")

[['Alabama', 14797010, 1], ['Alaska', 2793758, 3], ['Arizona', 23719079, 5], ['Arkansas', 9171935, 7], ['California', 189304472, 9], ['Colorado', 26587439, 11], ['Connecticut', 17158882, 13], ['Delaware', 4504805, 15], ['District of Columbia', 3285371, 17], ['Florida', 106471353, 19], ['Georgia', 39863068, 21], ['Hawaii', 5556071, 23], ['Idaho', 6336347, 25], ['Illinois', 44899268, 27], ['Indiana', 18127016, 29], ['Iowa', 9723702, 31], ['Kansas', 9509771, 33], ['Kentucky', 13198632, 35], ['Louisiana', 16923596, 37], ['Maine', 5468972, 39], ['Maryland', 23049084, 41], ['Massachusetts', 30907061, 43], ['Michigan', 32700692, 45], ['Minnesota', 19994802, 47], ['Mississippi', 9164153, 49], ['Missouri', 19463359, 51], ['Montana', 4414875, 53], ['Nebraska', 6305787, 55], ['Nevada', 12646359, 57], ['New  Hampshire', 6435512, 59], ['New Jersey', 44074085, 61], ['New Mexico', 5271452, 63], ['New York', 98252114, 65], ['North Carolina', 34400859, 67], ['North Dakota', 2948272, 69], ['Ohio', 36988

In [44]:
#session.query(nonemp).all()

In [46]:
# Join tables
merged_updated_results = [record for record in conn.execute(f"""
                                                     SELECT 
                                                       state_table.state_code,
                                                       state_table.percent_hs_grad,
                                                       state_table.per_capita_income,
                                                       crime_table.homicide_rate,
                                                       crime_table.firearms_death_rate,
                                                       crime_table.total_firearm_deaths,
                                                       household_table.household_size,
                                                       household_table.median_income,
                                                       poverty_table.sixteen_average,
                                                       poverty_table.sixteen_error,
                                                       poverty_table.fifteen_average,
                                                       poverty_table.fifteen_error,
                                                       poverty_table.seventeen_average,
                                                       poverty_table.seventeen_error,
                                                       poverty_table.three_year_error,
                                                       nonemp.county,
                                                       nonemp.NRCPTOT
                                                       
                                                     FROM state_table
                                                     
                                                     left join crime_table
                                                         on crime_table.state_code = state_table.state_code
                                                         
                                                     left join household_table
                                                         on household_table.state_code = state_table.state_code
                                                         
                                                     left join poverty_table
                                                         on poverty_table.state_code = state_table.state_code
                                                         
                                                     left join nonemp
                                                         on nonemp.state_code = state_table.state_code
                                                         
                                                     ORDER BY state_table.state_code 
                                                  """).fetchall()]
print(join_updated_results)

[('Alabama', 85.8, 26846, 8.3, 21.5, 1046, 2.55, 48486, 16.0, 1.5, 16.3, 1.9, 15.6, 1.6, 1.9, 14797010, 1), ('Alaska', 92.7, 35874, 8.4, 23.3, 177, 2.81, 76715, 12.2, 1.8, 10.9, 1.6, 12.6, 1.6, 2.0, 2793758, 3), ('Arizona', 86.8, 29265, 5.9, 15.2, 1094, 2.69, 56213, 14.4, 1.6, 16.6, 1.5, 13.2, 1.8, 2.0, 23719079, 5), ('Arkansas', 86.2, 25635, 8.6, 17.8, 541, 2.52, 45726, None, None, None, None, None, None, None, 9171935, 7), ('California', 82.9, 35021, 4.6, 7.9, 3184, 2.96, 71228, 12.5, 0.5, 13.9, 0.6, 12.0, 0.6, 0.8, 189304472, 9), ('Colorado', 91.4, 36415, 3.9, 14.3, 812, 2.56, 68811, 8.9, 0.9, 9.2, 1.4, 9.0, 1.1, 1.7, 26587439, 11), ('Connecticut', 90.5, 43056, 2.8, 4.6, 172, 2.54, 76106, 9.9, 1.3, 9.5, 1.4, 10.4, 1.4, 1.8, 17158882, 13), ('Delaware', 89.8, 33989, 5.6, 11.0, 111, 2.58, 65627, 9.6, 1.1, 11.3, 1.4, 8.1, 1.2, 1.8, 4504805, 15), ('District of Columbia', 90.6, 53321, None, None, None, 2.29, 82604, 14.9, 1.1, 16.5, 1.3, 14.3, 1.2, 1.7, 3285371, 17), ('Florida', 88.0, 3019

In [62]:
calculation_results_group_by = [record for record in conn.execute(f"""
                                                     SELECT
                                                       min(poverty_table.sixteen_average) as min_sixteen_average, 
                                                       max(poverty_table.sixteen_error) as max_sixteen_error,
                                                       min(poverty_table.fifteen_average) as min_fifteen_average,
                                                       max(poverty_table.fifteen_error) as max_fifteen_error,
                                                       min(poverty_table.seventeen_average) as min_seventeen_average,
                                                       max(poverty_table.seventeen_error) as max_seventeen_error,
                                                       avg(poverty_table.three_year_error) as avg_three_year_error,
                                                       avg(nonemp.county) as avg_county,
                                                       min(nonemp.NRCPTOT) as min_NRCPTOT
                                                       
                                                     FROM state_table
                                                     
                                                     left join crime_table
                                                         on crime_table.state_code = state_table.state_code
                                                    
                                                     left join household_table
                                                         on household_table.state_code = state_table.state_code

                                                     left join poverty_table
                                                         on poverty_table.state_code = state_table.state_code
                                                         
                                                     left join nonemp
                                                         on nonemp.state_code = state_table.state_code
                                                         
                                                     GROUP BY state_table.state_code
                                                     
                                                     
                                                  """).fetchall()][0]

#rounding
calculation_results_group_by = [a for a in calculation_results_group_by]

calculation_results_group_by = [round(a, 2) for a in calculation_results_group_by]
print(calculation_results_group_by)



[16.0, 1.5, 16.3, 1.9, 15.6, 1.6, 1.9, 14797010.0, 1]
